## Explorando dados do IBGE sobre leite do estado do Ceará e Quixadá

Instalando pacotes necessários

In [1]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


Importando pacotes necessários


In [51]:
import pandas as pd
import glob
import os

Carregando dados

In [52]:
caminho_pasta = 'planilhas/'

arquivos_caminhos = glob.glob(os.path.join(caminho_pasta, '*.xlsx'))
arquivos_para_padronizar = ["Estabelecimentos - Leite - tabela6912.xlsx", "Leite - tabela1086.xlsx"]

arquivos_iguais = [arq for arq in arquivos_caminhos if os.path.basename(arq) not in arquivos_para_padronizar]
arquivos_para_padronizar = [arq for arq in arquivos_caminhos if os.path.basename(arq) in arquivos_para_padronizar]


df_lista = []

for arquivo in arquivos_iguais:
    df = pd.read_excel(arquivo, sheet_name=None)

    for nome_folha, df_folha in df.items():
        if nome_folha != 'Notas':
            variavel = df_folha.iloc[0, 0].replace('Variável - ', '')
            anos = df_folha.iloc[2].reset_index(drop=True).dropna()
            valores_municipio = df_folha.iloc[4].reset_index(drop=True).dropna()
            municipio = valores_municipio.iloc[0]

            dict_temp = {
                'Variavel': variavel,
                'Ano': anos,
                'Municipio': municipio,
                'Valores_Municipio': valores_municipio[1:],
            }

            df_temp = pd.DataFrame(dict_temp)
            df_lista.append(df_temp)

arquivo_leite = pd.ExcelFile(arquivos_para_padronizar[0])
df_leite = pd.read_excel(arquivo_leite, sheet_name=None)

list_temp = []
for nome_folha, df_folha in df_leite.items():
    if nome_folha != 'Notas':
        variavel = df_folha.iloc[0, 0].replace('Variável - ', '')
        divisao_tempo = df_folha.iloc[2].reset_index(drop=True).dropna()
        valores_estado = df_folha.iloc[5].reset_index(drop=True).dropna()
        estado = valores_estado.iloc[0]

        for i, tempo in enumerate(divisao_tempo):
            trimestres, _, anos = tempo.split()
            trimestres = trimestres[:-1]

            dict_temp = {
                'Variavel': variavel,
                'Trimestre': trimestres,
                'Ano': anos,
                'Estado': estado,
                'Valores_Estado': valores_estado[i+2],
            }

            df_temp = pd.DataFrame([dict_temp])
            list_temp.append(df_temp)

df_leite = pd.concat(list_temp, ignore_index=True)
df_lista.append(df_leite)

arquivo_estabelecimentos = pd.ExcelFile(arquivos_para_padronizar[1])
df_estabelecimentos = pd.read_excel(arquivo_estabelecimentos, sheet_name=None)

for nome_folha, df_folha in df_estabelecimentos.items():
        if nome_folha != 'Notas':
            variavel = df_folha.iloc[0, 0].replace('Variável - ', '')
            anos = df_folha.iloc[2].reset_index(drop=True).dropna()
            valores_municipio = df_folha.iloc[5].reset_index(drop=True).dropna()
            municipio = valores_municipio.iloc[0]

            dict_temp = {
                'Variavel': variavel,
                'Ano': anos,
                'Municipio': municipio,
                'Valores_Municipio': valores_municipio[1:],
            }

            df_temp = pd.DataFrame(dict_temp).dropna()
            df_lista.append(df_temp)

Juntando dados e exportando em JSON

In [53]:

dados_padronizados = pd.concat(df_lista, ignore_index=True)
dados_padronizados = dados_padronizados.fillna('')
dados_padronizados.to_json('dados_padronizados.json', orient='records', force_ascii=False, indent=4)